In [1]:
import sys,re, os, glob, json, string, pprint
import pandas as pd
import numpy as np 
from snowballstemmer import TurkishStemmer 
from collections import Counter
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.tokenize import word_tokenize 
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix



## Read JSON

In [2]:
path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json" 
files = glob.glob(path)

ictihats = []
suc_array = []

        
top_ten_crimes = []
with open("data.json", encoding='utf-8') as jsonFile:
    data = json.load(jsonFile)
    top_ten_crimes = list(map(lambda x: x[0], data[:10]))
    

for file in files:
    f = open(file, "r", encoding='utf-8')
    jsonData = json.loads(f.read())
    key = jsonData["Suç"] if jsonData["Suç"] != '' else "other"
    if key in top_ten_crimes:
        ictihats.append(jsonData['ictihat'].strip())
        suc = jsonData["Suç"] if jsonData["Suç"] != '' else "other"
        suc_array.append(suc.strip())

        



## Create labels from dataset

In [3]:
# def has_key(dict, key): 
#     if key in dict.keys(): 
#         return 1 
#     return 0

# labels = {} 
# def readLabels(fileName): 
#     f = open(file, "r", encoding='utf-8') 
#     jsonData = json.loads(f.read())
#     key = jsonData["Suç"] if jsonData["Suç"] != '' else "other"
#     if (not has_key(labels, key)):
#         labels[key] = 1    
#     else :
#         labels[key] = labels[key] + 1

# path = "C://Users/onkol/Desktop/2021-01-20220322T055600Z-001/2021-01/*.json" 
# files = glob.glob(path)

# for file in files: 
#     readLabels(file)

# sort_orders = sorted(labels.items(), key=lambda x: x[1], reverse=True)

# jsonString = json.dumps(sort_orders, indent = 4, ensure_ascii=False)
# jsonFile = open("data.json", "w", encoding='utf-8')
# jsonFile.write(jsonString)
# jsonFile.close()



## Preprocessing

In [4]:
ictihats_lower = []
for i in ictihats:
    i = i.lower()
    ictihats_lower.append(i)
    

ictihats_punctuation = []

file = open("stopwords.txt", "r", newline='', encoding='utf-8')
result = file.read()
stopwords = word_tokenize(result)
word_size = 20

for ictihat in ictihats_lower:
    ictihat = ictihat.translate(str.maketrans("", "", string.punctuation))
    #print(ictihat)
    tokenized_words = ([word for word in ictihat.split() if word not in stopwords and len(word) > 1 and re.match("^[A-Z0-9a-zğüşöçİĞÜŞÖÇ]*$", word)])
    #print(tokenized_words)
    ictihats_punctuation.append(' '.join(tokenized_words[:word_size]))
    



# print(ictihats_punctuation[:10])   
# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#     preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# print(preprocessed_ictihats)
turkStem = TurkishStemmer()
for index, ictihat in enumerate(ictihats_punctuation,start=0):
    #print([turkStem.stemWord(word) for word in ictihat])
    ictihats_punctuation[index] = ' '.join([turkStem.stemWord(word) for word in ictihat.split()])
    
#print(ictihats_punctuation)

## write data into csv file

In [5]:
df = pd.DataFrame(columns = ['ictihats', 'sucs'])
df['ictihats'] = ictihats_punctuation
df['sucs'] = suc_array
csv_file_name = 'train_set_' + str(word_size) + '.csv'

df.to_csv(csv_file_name, encoding='utf-8', index=False)


## Split Dataset

In [6]:
# my_csv = pd.read_csv(filename)
# column = csv_file_name[]
# csv_file_name = 'train_set_' + str(word_size) + '.csv'
# my_csv = pd.read_csv(csv_file_name)
# #print(my_csv.columns)
# x = my_csv['ictihats']
# print(x.tolist())
# y = my_csv['sucs']
# x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.20,random_state=36)
#print(x_train)

x_train, x_test, y_train, y_test = train_test_split(ictihats_punctuation, 
                                                    suc_array, 
                                                    test_size=0.20,random_state=36)

## Vectorize

In [7]:
tfidfvectorizer = TfidfVectorizer(analyzer='word',stop_words= stopwords)
training_data = tfidfvectorizer.fit_transform(x_train)
testing_data = tfidfvectorizer.transform(x_test)


C:\Users\onkol\anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:388: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['lahî'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


## Support Vector Machines (specifically linear SVM)

In [8]:
svclassifier = SVC(kernel='linear')
svclassifier.fit(training_data, y_train)
y_pred = svclassifier.predict(testing_data)

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))

[[  78    0    0    0    0    0    0    0    0    0]
 [   0   14    0    1    0    0    0   71    0    0]
 [   0    0  133    0    1    0    0    0    0    0]
 [   0    0    4  284    0    0    0    1    0    1]
 [   0    0    0    0   62    0    0    0    4    0]
 [   0    2    0    0    0   53    0    0    0    0]
 [   0    0    0    0    0    0   62    0    0    0]
 [   0    3    3    2    0    0    0  103    0    0]
 [   0    0    0    0    0    0    0    0  133    1]
 [   0    0    2    1    0    1    2    1    0 2593]]
                                            precision    recall  f1-score   support

              5607 sayılı Kanuna muhalefet       1.00      1.00      1.00        78
                                   Hakaret       0.74      0.16      0.27        86
                                 Hırsızlık       0.94      0.99      0.96       134
                           Kasten yaralama       0.99      0.98      0.98       290
Kullanmak için uyuşturucu madde bulundurma      

## Multinomial Naive Bayes

In [9]:
naive_bayes = MultinomialNB()
naive_bayes.fit(training_data, y_train)

predictions = naive_bayes.predict(testing_data)

# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
# print('Precision score: ', format(precision_score(y_test, predictions, average='weighted')))
# print('Recall score: ', format(recall_score(y_test, predictions ,average='weighted')))
# print('F1 score: ', format(f1_score(y_test, predictions ,average='weighted')))

print(confusion_matrix(y_test,predictions))
print(classification_report(y_test,predictions))


[[  75    0    0    0    0    0    0    0    0    3]
 [   0    0    0    0    0    0    0   64    0   22]
 [   0    0   82    6    0    0    0    0    0   46]
 [   0    0    1  256    0    0    0    0    0   33]
 [   0    0    0    0   12    0    0    0    6   48]
 [   0    0    0    0    0   50    0    0    0    5]
 [   0    0    0    2    0    0   52    0    0    8]
 [   0    0    1    4    0    0    0   90    0   16]
 [   0    0    0    1    0    0    0    0   37   96]
 [   2    0    0    1    0    1    0    1    0 2595]]
                                            precision    recall  f1-score   support

              5607 sayılı Kanuna muhalefet       0.97      0.96      0.97        78
                                   Hakaret       0.00      0.00      0.00        86
                                 Hırsızlık       0.98      0.61      0.75       134
                           Kasten yaralama       0.95      0.88      0.91       290
Kullanmak için uyuşturucu madde bulundurma      

C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\onkol\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


## Logistic Regression

In [10]:
log_regres_classifier = LogisticRegression(random_state = 36)
log_regres_classifier.fit(training_data, y_train)
y_pred = log_regres_classifier.predict(testing_data)
cm = confusion_matrix(y_test, y_pred)


# print('Accuracy score: ', (accuracy_score(y_test, y_pred)))
# print('Precision score: ', (precision_score(y_test, y_pred, average='weighted')))
# print('Recall score: ', (recall_score(y_test, y_pred ,average='weighted')))
# print('F1 score: ', (f1_score(y_test, y_pred ,average='weighted')))
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))


[[  77    0    0    0    0    0    0    0    0    1]
 [   0   12    1    1    0    0    0   71    0    1]
 [   0    0  130    0    1    0    0    0    0    3]
 [   0    0    4  284    0    0    0    1    0    1]
 [   0    0    1    0   56    0    0    0    9    0]
 [   0    2    0    0    0   50    0    0    2    1]
 [   0    0    0    0    0    0   62    0    0    0]
 [   0    3    4    2    0    0    0  102    0    0]
 [   0    0    0    0    1    0    0    0  131    2]
 [   0    0    1    4    0    1    1    1    0 2592]]
                                            precision    recall  f1-score   support

              5607 sayılı Kanuna muhalefet       1.00      0.99      0.99        78
                                   Hakaret       0.71      0.14      0.23        86
                                 Hırsızlık       0.92      0.97      0.95       134
                           Kasten yaralama       0.98      0.98      0.98       290
Kullanmak için uyuşturucu madde bulundurma      

In [11]:
#print(len(suc_array))
#print(ictihats_punctuation)
# index_list = []
# for i in range(0,100):
#     index_list.append(str(i))

# preprocessed_ictihats = []
# for i in ictihats_punctuation:
#        preprocessed_ictihats.append(list(filter(lambda x: x != "",i.split(" "))))

# #print(preprocessed_ictihats)
# turkStem = TurkishStemmer()
# for index, ictihat in enumerate(preprocessed_ictihats,start=0):
#     preprocessed_ictihats[index] = [turkStem.stemWord(word) for word in ictihat]
    
#print(preprocessed_ictihats)

#tfidf_wm_array
#tfidf_wm_array.shape
# frequency_matrix = pd.DataFrame(tfidf_wm_array, columns=tfidfvectorizer.get_feature_names())
# from sklearn.model_selection import train_test_split
# y = np.array([0,1,2,3,4,5,6,7,8,9,10]) 


# naive_bayes = MultinomialNB()
# naive_bayes.fit(training_data, y_train)

# predictions = naive_bayes.predict(testing_data)

# from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
# print('Accuracy score: ', format(accuracy_score(y_test, predictions)))
# print('Precision score: ', format(precision_score(y_test, predictions,
#                                            average='macro')))
# print('Recall score: ', format(recall_score(y_test, predictions
#                                            ,average='macro')))
# print('F1 score: ', format(f1_score(y_test, predictions
#                                            ,average='macro')))

# print(confusion_matrix(y_test,predictions))


# from sklearn.svm import SVC
# svclassifier = SVC(kernel='linear')
# svclassifier.fit(training_data, y_train)
# y_pred = svclassifier.predict(testing_data)

# from sklearn.metrics import classification_report, confusion_matrix
# print(confusion_matrix(y_test,y_pred))
# print(classification_report(y_test,y_pred))



# frequency_matrix
# print(tfidfvectorizer.vocabulary_)
# print(tfidf_wm.toarray())
# tfidf_tokens = tfidfvectorizer.get_feature_names()
# df_tfidfvect = pd.DataFrame(data = tfidf_wm.toarray(),index = index_list,columns = tfidf_tokens)
# print("\nTD-IDF Vectorizer\n")
# print(df_tfidfvect)
# vectorizer.get_feature_names_out()
# ictihat_freq = []


# for i in preprocessed_ictihats:
#     count_freq = Counter(i)
#     ictihat_freq.append(count_freq)
    
# pprint.pprint(ictihat_freq)

